In [294]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brinkley97/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read Data

In [295]:
dataset = "../datasets/amazon_reviews_us_Office_Products_v1_00.tsv"
amazon_reviews_copy_df = pd.read_csv(dataset, sep='\t', on_bad_lines='skip', low_memory=False)

In [296]:
# amazon_reviews_copy_df = amazon_reviews_df.copy()

## Keep Reviews and Ratings

In [297]:
# reviews_ratings_df = amazon_reviews_copy_df[['star_rating', 'review_headline', 'review_body']]
# reviews_ratings_df.reset_index(drop=True)
# reviews_ratings_df

 ## We form three classes and select 20000 reviews randomly from each class.
- [ ] 100,000 each


In [298]:
reviews_ratings_df = amazon_reviews_copy_df[['star_rating', 'review_headline', 'review_body']]
reviews_ratings_df.reset_index(drop=True)

valid = ["1","2","3","4","5"]
from copy import deepcopy

stars = deepcopy(reviews_ratings_df).star_rating.astype(str) # turn entries to strings
where_valid = stars.index[stars.isin(valid)].tolist() # check valid list and see which of our stars match
reviews_ratings_df = reviews_ratings_df.iloc[where_valid]

reviews_ratings_df

,star_rating,review_headline,review_body
0,5,Five Stars,Great product.
1,5,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...
2,5,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it."
3,1,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...
4,4,Four Stars,Gorgeous colors and easy to use
...,...,...,...
2640249,4,Great value! A must if you hate to carry thing...,I can't live anymore whithout my Palm III. But...
2640250,4,Attaches the Palm Pilot like an appendage,Although the Palm Pilot is thin and compact it...
2640251,4,"Excellent information, pictures and stories, I...",This book had a lot of great content without b...
2640252,5,class text,I am teaching a course in Excel and am using t...


In [299]:
reviews_ratings_df.star_rating = reviews_ratings_df.star_rating.apply(lambda x: int(x))

In [300]:
reviews_ratings_df

,star_rating,review_headline,review_body
0,5,Five Stars,Great product.
1,5,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...
2,5,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it."
3,1,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...
4,4,Four Stars,Gorgeous colors and easy to use
...,...,...,...
2640249,4,Great value! A must if you hate to carry thing...,I can't live anymore whithout my Palm III. But...
2640250,4,Attaches the Palm Pilot like an appendage,Although the Palm Pilot is thin and compact it...
2640251,4,"Excellent information, pictures and stories, I...",This book had a lot of great content without b...
2640252,5,class text,I am teaching a course in Excel and am using t...


In [301]:

def separate_reviews_by_rating(df: pd.DataFrame, rating_col: str, threshold: int, sentiment_type: str):


    if sentiment_type == 'positive_sentiment':
        positive_review_threshold = df[rating_col] > threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 1

    elif sentiment_type == 'negative_sentiment':
        positive_review_threshold = df[rating_col] < threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 0

    elif sentiment_type == 'neutral_sentiment':
        positive_review_threshold = df[rating_col] == threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 3
        
    return df

positive_sentiment_df = separate_reviews_by_rating(reviews_ratings_df, 'star_rating', 3, 'positive_sentiment')
negative_sentiment_df = separate_reviews_by_rating(reviews_ratings_df, 'star_rating', 3, 'negative_sentiment')
neutral_sentiment_df = separate_reviews_by_rating(reviews_ratings_df, 'star_rating', 3, 'neutral_sentiment')
# separate_reviews_by_rating(reviews_ratings_df, 'star_rating', 3, 'n_sentiment')

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_29073/915735845.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 1
/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_29073/915735845.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 0
/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_29073/915735845.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [302]:
# positive_sentiment_df

In [303]:
# positive_sentiment_df["star_rating"].value_counts()

In [304]:
# negative_sentiment_df

In [305]:
# negative_sentiment_df["star_rating"].value_counts()

In [306]:
# neutral_sentiment_df

In [307]:
# neutral_sentiment_df["star_rating"].value_counts()

In [308]:
pos_rand_sampled_df = positive_sentiment_df.sample(100000)
pos_rand_sampled_df

,star_rating,review_headline,review_body,positive_sentiment
1121075,5,Five Stars,Love these colors!,1
936358,5,Five Stars,Easy to put paper instead of everywhere,1
2432735,5,Just loved it!!!,Just Loved it!!! Need to master it a little bi...,1
85646,5,Great kit for schoolers!!,I got it for my niece and she love it because ...,1
2478133,5,"Good price, arrived fast",Received the cartridges pictured - and it arri...,1
...,...,...,...,...
363245,5,I love it! A purrfect mouse pad,I love it! A purrfect mouse pad.,1
339204,5,Five Stars,Great price; Exceptional quality! Satisfied s...,1
2265415,4,Good scanner,I needed to replace my original scanner that w...,1
2269008,4,Hi,The laser pointer is green and bright.it is a ...,1


In [309]:
neg_rand_sampled_df = negative_sentiment_df.sample(100000)
neg_rand_sampled_df

,star_rating,review_headline,review_body,negative_sentiment
795949,2,"The paper is perfect, but the online templates...","The paper is perfect, but the online templates...",0
2062966,1,Instalation not easy,"Maybe it is me, but major process wtih Google ...",0
1176532,1,One Star,Printer says not Epson cartridges. They must b...,0
1442658,1,WONT LAST! I WANT A REFUND!,Got it brand new here. I have printed about 40...,0
2290236,1,Terrible Experience,Color Ink worked after reinstalling it 3 times...,0
...,...,...,...,...
393612,1,Hmmm ????,Why brag about these cartridges as &#34;CHIP ...,0
1371670,1,Didn't last very longâ€¦ and now my printer is...,"Got this a couple weeks ago, direct from Amazo...",0
2551306,1,A Lousy Printer,I have this printer on my office desk. I hate ...,0
2287493,1,Avoid this seller if at all possible,I purchased these in April 2011 as backups for...,0


In [310]:
# reviews_ratings_df = pd.concat([pos_rand_sampled_df, neg_rand_sampled_df])

In [311]:
reviews_ratings_with_sentiment_df = pd.concat([pos_rand_sampled_df, neg_rand_sampled_df])

In [312]:
reviews_ratings_with_sentiment_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment
1121075,5,Five Stars,Love these colors!,1.0,NaN
936358,5,Five Stars,Easy to put paper instead of everywhere,1.0,NaN
2432735,5,Just loved it!!!,Just Loved it!!! Need to master it a little bi...,1.0,NaN
85646,5,Great kit for schoolers!!,I got it for my niece and she love it because ...,1.0,NaN
2478133,5,"Good price, arrived fast",Received the cartridges pictured - and it arri...,1.0,NaN
...,...,...,...,...,...
393612,1,Hmmm ????,Why brag about these cartridges as &#34;CHIP ...,NaN,0.0
1371670,1,Didn't last very longâ€¦ and now my printer is...,"Got this a couple weeks ago, direct from Amazo...",NaN,0.0
2551306,1,A Lousy Printer,I have this printer on my office desk. I hate ...,NaN,0.0
2287493,1,Avoid this seller if at all possible,I purchased these in April 2011 as backups for...,NaN,0.0


# Data Cleaning

## Lower case
- NOTE: Not all reviews are a string. To solve,
    - [ ] Filter out non-strings when/before randomly sampling

In [313]:
def convert_reviews_to_lower_case(df: pd.DataFrame, col_name: str):
    """Convert all reviews to lower case

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the lower cased reviews
    """
    
    lower_case_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    
    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_reviews_idx, type(text_review), text_review)

        if type(text_review) != str:
            converted_str = str(text_review)
            # update_text_review = converted_str.lower()
            lower_case_reviews.append(update_text_review)
            # print(text_reviews_idx, update_text_review)
            # print()
        else:
            update_text_review = text_review.lower()
            lower_case_reviews.append(update_text_review)
            # print(text_reviews_idx, update_text_review)
            # print()

    update_col_name = col_name + '_lower_cased'
    updated_df[update_col_name] = lower_case_reviews
    
    # updated_df = df
    return updated_df

In [314]:
# reviews_ratings_with_sentiment_df

In [315]:
reviews_lc_hl_df = convert_reviews_to_lower_case(reviews_ratings_with_sentiment_df, 'review_headline')

In [316]:
# reviews_lc_hl_df.head(10)

In [317]:
reviews_lc_rb_df = convert_reviews_to_lower_case(reviews_lc_hl_df, 'review_body')

In [318]:
reviews_lc_rb_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1121075,5,Five Stars,Love these colors!,1.0,NaN,five stars,love these colors!
936358,5,Five Stars,Easy to put paper instead of everywhere,1.0,NaN,five stars,easy to put paper instead of everywhere
2432735,5,Just loved it!!!,Just Loved it!!! Need to master it a little bi...,1.0,NaN,just loved it!!!,just loved it!!! need to master it a little bi...
85646,5,Great kit for schoolers!!,I got it for my niece and she love it because ...,1.0,NaN,great kit for schoolers!!,i got it for my niece and she love it because ...
2478133,5,"Good price, arrived fast",Received the cartridges pictured - and it arri...,1.0,NaN,"good price, arrived fast",received the cartridges pictured - and it arri...
...,...,...,...,...,...,...,...
393612,1,Hmmm ????,Why brag about these cartridges as &#34;CHIP ...,NaN,0.0,hmmm ????,why brag about these cartridges as &#34;chip ...
1371670,1,Didn't last very longâ€¦ and now my printer is...,"Got this a couple weeks ago, direct from Amazo...",NaN,0.0,didn't last very longâ€¦ and now my printer is...,"got this a couple weeks ago, direct from amazo..."
2551306,1,A Lousy Printer,I have this printer on my office desk. I hate ...,NaN,0.0,a lousy printer,i have this printer on my office desk. i hate ...
2287493,1,Avoid this seller if at all possible,I purchased these in April 2011 as backups for...,NaN,0.0,avoid this seller if at all possible,i purchased these in april 2011 as backups for...


## Remove HTML and URLs
- [ ] Verify by finding a specific entry with HTML, URL.

In [319]:
def remove_html_and_urls(df:pd.DataFrame, col_name: str):
    """Remove HTML and URLs from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the html_and_urls removed
    """
    
    # url_pattern = re.compile(r'https?://\S+|www\. \S+')

    # updated_reviews = []
    # updated_df = df.copy()
    # text_reviews = df[col_name].values

    # for text_reviews_idx in range(len(text_reviews)):
    #     text_review = text_reviews[text_reviews_idx]

    #     if type(text_review) != str:
    #         updated_reviews.append(text_review)
    #     else:
    #         update_text_review = url_pattern.sub(r'', text_review)
    #         updated_reviews.append(update_text_review)

    # updated_df[col_name] = updated_reviews
    # return updated_df

    url_pattern = re.compile(r'https?://\S+|www\.\S+')

    def clean_text(text):
        if isinstance(text, str):
            return url_pattern.sub('', text)
        return text

    df[col_name] = df[col_name].apply(clean_text)
    return df

In [320]:
reviews_no_html_urls_df = remove_html_and_urls(reviews_lc_rb_df, 'review_headline')

In [321]:
# reviews_no_html_urls_df

In [322]:
reviews_no_html_urls_rb_df = remove_html_and_urls(reviews_no_html_urls_df, 'review_body')

In [323]:
reviews_no_html_urls_rb_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1121075,5,Five Stars,Love these colors!,1.0,NaN,five stars,love these colors!
936358,5,Five Stars,Easy to put paper instead of everywhere,1.0,NaN,five stars,easy to put paper instead of everywhere
2432735,5,Just loved it!!!,Just Loved it!!! Need to master it a little bi...,1.0,NaN,just loved it!!!,just loved it!!! need to master it a little bi...
85646,5,Great kit for schoolers!!,I got it for my niece and she love it because ...,1.0,NaN,great kit for schoolers!!,i got it for my niece and she love it because ...
2478133,5,"Good price, arrived fast",Received the cartridges pictured - and it arri...,1.0,NaN,"good price, arrived fast",received the cartridges pictured - and it arri...
...,...,...,...,...,...,...,...
393612,1,Hmmm ????,Why brag about these cartridges as &#34;CHIP ...,NaN,0.0,hmmm ????,why brag about these cartridges as &#34;chip ...
1371670,1,Didn't last very longâ€¦ and now my printer is...,"Got this a couple weeks ago, direct from Amazo...",NaN,0.0,didn't last very longâ€¦ and now my printer is...,"got this a couple weeks ago, direct from amazo..."
2551306,1,A Lousy Printer,I have this printer on my office desk. I hate ...,NaN,0.0,a lousy printer,i have this printer on my office desk. i hate ...
2287493,1,Avoid this seller if at all possible,I purchased these in April 2011 as backups for...,NaN,0.0,avoid this seller if at all possible,i purchased these in april 2011 as backups for...


## Remove Non-alphabetical characters
- [ ] If entry has no letters, leave blank?

In [324]:
def remove_non_alphabetical_characters(df:pd.DataFrame, col_name: str):
    """Remove Non-alphabetical characters from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the non-alphabetical characters removed
    """
    


    updated_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_review)

        # if type(text_review) != str:
        #     updated_reviews.append(text_review)
        # else:
        update_text_review = re.sub(r'[^a-zA-Z\s]', '', text_review)
        # print(text_review)
        updated_reviews.append(update_text_review)

    updated_df[col_name] = updated_reviews
    return updated_df

In [325]:
reviews_alph_chars_df = remove_non_alphabetical_characters(reviews_no_html_urls_rb_df, 'review_headline_lower_cased')

In [326]:
reviews_alph_chars_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1121075,5,Five Stars,Love these colors!,1.0,NaN,five stars,love these colors!
936358,5,Five Stars,Easy to put paper instead of everywhere,1.0,NaN,five stars,easy to put paper instead of everywhere
2432735,5,Just loved it!!!,Just Loved it!!! Need to master it a little bi...,1.0,NaN,just loved it,just loved it!!! need to master it a little bi...
85646,5,Great kit for schoolers!!,I got it for my niece and she love it because ...,1.0,NaN,great kit for schoolers,i got it for my niece and she love it because ...
2478133,5,"Good price, arrived fast",Received the cartridges pictured - and it arri...,1.0,NaN,good price arrived fast,received the cartridges pictured - and it arri...
...,...,...,...,...,...,...,...
393612,1,Hmmm ????,Why brag about these cartridges as &#34;CHIP ...,NaN,0.0,hmmm,why brag about these cartridges as &#34;chip ...
1371670,1,Didn't last very longâ€¦ and now my printer is...,"Got this a couple weeks ago, direct from Amazo...",NaN,0.0,didnt last very long and now my printer is fin...,"got this a couple weeks ago, direct from amazo..."
2551306,1,A Lousy Printer,I have this printer on my office desk. I hate ...,NaN,0.0,a lousy printer,i have this printer on my office desk. i hate ...
2287493,1,Avoid this seller if at all possible,I purchased these in April 2011 as backups for...,NaN,0.0,avoid this seller if at all possible,i purchased these in april 2011 as backups for...


In [327]:
reviews_alph_chars_rb_df = remove_non_alphabetical_characters(reviews_alph_chars_df, 'review_body_lower_cased')

In [328]:
reviews_alph_chars_rb_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1121075,5,Five Stars,Love these colors!,1.0,NaN,five stars,love these colors
936358,5,Five Stars,Easy to put paper instead of everywhere,1.0,NaN,five stars,easy to put paper instead of everywhere
2432735,5,Just loved it!!!,Just Loved it!!! Need to master it a little bi...,1.0,NaN,just loved it,just loved it need to master it a little bit s...
85646,5,Great kit for schoolers!!,I got it for my niece and she love it because ...,1.0,NaN,great kit for schoolers,i got it for my niece and she love it because ...
2478133,5,"Good price, arrived fast",Received the cartridges pictured - and it arri...,1.0,NaN,good price arrived fast,received the cartridges pictured and it arriv...
...,...,...,...,...,...,...,...
393612,1,Hmmm ????,Why brag about these cartridges as &#34;CHIP ...,NaN,0.0,hmmm,why brag about these cartridges as chip inclu...
1371670,1,Didn't last very longâ€¦ and now my printer is...,"Got this a couple weeks ago, direct from Amazo...",NaN,0.0,didnt last very long and now my printer is fin...,got this a couple weeks ago direct from amazon...
2551306,1,A Lousy Printer,I have this printer on my office desk. I hate ...,NaN,0.0,a lousy printer,i have this printer on my office desk i hate i...
2287493,1,Avoid this seller if at all possible,I purchased these in April 2011 as backups for...,NaN,0.0,avoid this seller if at all possible,i purchased these in april as backups for my ...


## Remove extra spaces
- [ ] Verify with a specific entry

In [329]:
def remove_extra_spaces(df:pd.DataFrame, col_name: str):
    """Remove extra spaces from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    updated_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_review)

        # if type(text_review) != str:
        #     updated_reviews.append(text_review)
        # else:
        update_text_review = re.sub(r' +', ' ', text_review)
        # print(text_review)
        updated_reviews.append(update_text_review)

    updated_df[col_name] = updated_reviews
    return updated_df

In [330]:
reviews_no_extra_space_df = remove_extra_spaces(reviews_alph_chars_rb_df, 'review_headline_lower_cased')

In [331]:
# reviews_no_extra_space_df

In [332]:
reviews_no_extra_space_rb_df = remove_extra_spaces(reviews_no_extra_space_df, 'review_body_lower_cased')

In [333]:
reviews_no_extra_space_rb_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1121075,5,Five Stars,Love these colors!,1.0,NaN,five stars,love these colors
936358,5,Five Stars,Easy to put paper instead of everywhere,1.0,NaN,five stars,easy to put paper instead of everywhere
2432735,5,Just loved it!!!,Just Loved it!!! Need to master it a little bi...,1.0,NaN,just loved it,just loved it need to master it a little bit s...
85646,5,Great kit for schoolers!!,I got it for my niece and she love it because ...,1.0,NaN,great kit for schoolers,i got it for my niece and she love it because ...
2478133,5,"Good price, arrived fast",Received the cartridges pictured - and it arri...,1.0,NaN,good price arrived fast,received the cartridges pictured and it arrive...
...,...,...,...,...,...,...,...
393612,1,Hmmm ????,Why brag about these cartridges as &#34;CHIP ...,NaN,0.0,hmmm,why brag about these cartridges as chip includ...
1371670,1,Didn't last very longâ€¦ and now my printer is...,"Got this a couple weeks ago, direct from Amazo...",NaN,0.0,didnt last very long and now my printer is fin...,got this a couple weeks ago direct from amazon...
2551306,1,A Lousy Printer,I have this printer on my office desk. I hate ...,NaN,0.0,a lousy printer,i have this printer on my office desk i hate i...
2287493,1,Avoid this seller if at all possible,I purchased these in April 2011 as backups for...,NaN,0.0,avoid this seller if at all possible,i purchased these in april as backups for my n...


## Remove Contractions
- [ ] Need to update; make my own

In [334]:
contraction_mapping = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "don't": "do not"
}

In [335]:
def expand_contractions(input_text):
    # Function to replace contractions with their expanded forms
    def replace(match):
        print("match:", match)
        return contraction_mapping[match.group(0)]

    # Use regular expression to find contractions and replace them
    contraction_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE | re.DOTALL)
    expanded_text = contraction_pattern.sub(replace, input_text)

    return expanded_text

In [336]:
def remove_contractions(df:pd.DataFrame, col_name: str):
    """Remove contractions from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    updated_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_review)

        # if type(text_review) != str:
        #     updated_reviews.append(text_review)
        # else:
        without_contraction = expand_contractions(text_review)
        # print(text_review)
        updated_reviews.append(without_contraction)

    updated_df[col_name] = updated_reviews
    return updated_df

In [337]:
reviews_no_contractions_df = remove_contractions(reviews_alph_chars_rb_df, 'review_headline_lower_cased')

In [338]:
# reviews_no_contractions_df

In [339]:
reviews_no_contractions_rb_df = remove_contractions(reviews_no_contractions_df, 'review_body_lower_cased')

In [340]:
reviews_no_contractions_rb_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1121075,5,Five Stars,Love these colors!,1.0,NaN,five stars,love these colors
936358,5,Five Stars,Easy to put paper instead of everywhere,1.0,NaN,five stars,easy to put paper instead of everywhere
2432735,5,Just loved it!!!,Just Loved it!!! Need to master it a little bi...,1.0,NaN,just loved it,just loved it need to master it a little bit s...
85646,5,Great kit for schoolers!!,I got it for my niece and she love it because ...,1.0,NaN,great kit for schoolers,i got it for my niece and she love it because ...
2478133,5,"Good price, arrived fast",Received the cartridges pictured - and it arri...,1.0,NaN,good price arrived fast,received the cartridges pictured and it arriv...
...,...,...,...,...,...,...,...
393612,1,Hmmm ????,Why brag about these cartridges as &#34;CHIP ...,NaN,0.0,hmmm,why brag about these cartridges as chip inclu...
1371670,1,Didn't last very longâ€¦ and now my printer is...,"Got this a couple weeks ago, direct from Amazo...",NaN,0.0,didnt last very long and now my printer is fin...,got this a couple weeks ago direct from amazon...
2551306,1,A Lousy Printer,I have this printer on my office desk. I hate ...,NaN,0.0,a lousy printer,i have this printer on my office desk i hate i...
2287493,1,Avoid this seller if at all possible,I purchased these in April 2011 as backups for...,NaN,0.0,avoid this seller if at all possible,i purchased these in april as backups for my ...


# Pre-processing

## remove the stop words 

In [341]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [342]:
def filter_stop_words(df:pd.DataFrame, col_name: str):
    """Filter stop words out from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    updated_reviews = []
    
    updated_df = df.copy()
    text_reviews = df[col_name].values

    stop_words = set(stopwords.words("english"))

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        text_review_words = word_tokenize(text_review) 

        filtered_review = []

        for text_review_words_idx in range(len(text_review_words)):
            text_review_word = text_review_words[text_review_words_idx]

            if text_review_word not in stop_words:
                filtered_review.append(text_review_word)

        filtered_review = " ".join(filtered_review)
        updated_reviews.append(filtered_review)

    updated_df[col_name] = updated_reviews
    return updated_df

In [343]:
without_stop_words_rb_df = filter_stop_words(reviews_no_contractions_rb_df, 'review_body_lower_cased')

In [344]:
without_stop_words_rb_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1121075,5,Five Stars,Love these colors!,1.0,NaN,five stars,love colors
936358,5,Five Stars,Easy to put paper instead of everywhere,1.0,NaN,five stars,easy put paper instead everywhere
2432735,5,Just loved it!!!,Just Loved it!!! Need to master it a little bi...,1.0,NaN,just loved it,loved need master little bit since im cutting ...
85646,5,Great kit for schoolers!!,I got it for my niece and she love it because ...,1.0,NaN,great kit for schoolers,got niece love little compact kit fit perfect ...
2478133,5,"Good price, arrived fast",Received the cartridges pictured - and it arri...,1.0,NaN,good price arrived fast,received cartridges pictured arrived within co...
...,...,...,...,...,...,...,...
393612,1,Hmmm ????,Why brag about these cartridges as &#34;CHIP ...,NaN,0.0,hmmm,brag cartridges chip included show ink level h...
1371670,1,Didn't last very longâ€¦ and now my printer is...,"Got this a couple weeks ago, direct from Amazo...",NaN,0.0,didnt last very long and now my printer is fin...,got couple weeks ago direct amazon hp brand ne...
2551306,1,A Lousy Printer,I have this printer on my office desk. I hate ...,NaN,0.0,a lousy printer,printer office desk hate heres whybr br takes ...
2287493,1,Avoid this seller if at all possible,I purchased these in April 2011 as backups for...,NaN,0.0,avoid this seller if at all possible,purchased april backups new nw original black ...


## perform lemmatization  

- "I was jogging with Aman for 3 miles"
- "I was jog with Aman for 3 miles"

In [345]:
from nltk.stem import WordNetLemmatizer

def lemmentize_review(df:pd.DataFrame, col_name: str):
    """Lemmentize all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    updated_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    
    lem = WordNetLemmatizer()

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print("Review: ", text_review)
        
        text_review_words = word_tokenize(text_review) 

        lemmed_sentence = []

        for text_review_words_idx in range(len(text_review_words)): # get each word in the string review
            text_review_word = text_review_words[text_review_words_idx]
            lemmed_word = lem.lemmatize(text_review_word)
            lemmed_sentence.append(lemmed_word)
            filtered_review = " ".join(lemmed_sentence)
    
        # print("New sentence", filtered_review)
        # print()

        updated_reviews.append(filtered_review)

    updated_df[col_name] = updated_reviews
    return updated_df

In [346]:
lemmed_df = lemmentize_review(without_stop_words_rb_df, 'review_body_lower_cased')

In [347]:
lemmed_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1121075,5,Five Stars,Love these colors!,1.0,NaN,five stars,love color
936358,5,Five Stars,Easy to put paper instead of everywhere,1.0,NaN,five stars,easy put paper instead everywhere
2432735,5,Just loved it!!!,Just Loved it!!! Need to master it a little bi...,1.0,NaN,just loved it,loved need master little bit since im cutting ...
85646,5,Great kit for schoolers!!,I got it for my niece and she love it because ...,1.0,NaN,great kit for schoolers,got niece love little compact kit fit perfect ...
2478133,5,"Good price, arrived fast",Received the cartridges pictured - and it arri...,1.0,NaN,good price arrived fast,received cartridge pictured arrived within cou...
...,...,...,...,...,...,...,...
393612,1,Hmmm ????,Why brag about these cartridges as &#34;CHIP ...,NaN,0.0,hmmm,brag cartridge chip included show ink level hp...
1371670,1,Didn't last very longâ€¦ and now my printer is...,"Got this a couple weeks ago, direct from Amazo...",NaN,0.0,didnt last very long and now my printer is fin...,got couple week ago direct amazon hp brand new...
2551306,1,A Lousy Printer,I have this printer on my office desk. I hate ...,NaN,0.0,a lousy printer,printer office desk hate here whybr br take lo...
2287493,1,Avoid this seller if at all possible,I purchased these in April 2011 as backups for...,NaN,0.0,avoid this seller if at all possible,purchased april backup new nw original black t...


In [348]:
pos_sentiment = lemmed_df['positive_sentiment'].dropna()
pos_sentiment

1121075    1.0
936358     1.0
2432735    1.0
85646      1.0
2478133    1.0
          ... 
363245     1.0
339204     1.0
2265415    1.0
2269008    1.0
931688     1.0
Name: positive_sentiment, Length: 100000, dtype: float64

In [349]:
neg_sentiment = lemmed_df['negative_sentiment'].dropna()
neg_sentiment

795949     0.0
2062966    0.0
1176532    0.0
1442658    0.0
2290236    0.0
          ... 
393612     0.0
1371670    0.0
2551306    0.0
2287493    0.0
2044301    0.0
Name: negative_sentiment, Length: 100000, dtype: float64

In [350]:
lemmed_df['sentiment'] = pd.concat([pos_sentiment, neg_sentiment])

In [351]:
lemmed_df['sentiment'].unique()

array([1., 0.])

In [352]:
lemmed_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased,sentiment
1121075,5,Five Stars,Love these colors!,1.0,NaN,five stars,love color,1.0
936358,5,Five Stars,Easy to put paper instead of everywhere,1.0,NaN,five stars,easy put paper instead everywhere,1.0
2432735,5,Just loved it!!!,Just Loved it!!! Need to master it a little bi...,1.0,NaN,just loved it,loved need master little bit since im cutting ...,1.0
85646,5,Great kit for schoolers!!,I got it for my niece and she love it because ...,1.0,NaN,great kit for schoolers,got niece love little compact kit fit perfect ...,1.0
2478133,5,"Good price, arrived fast",Received the cartridges pictured - and it arri...,1.0,NaN,good price arrived fast,received cartridge pictured arrived within cou...,1.0
...,...,...,...,...,...,...,...,...
393612,1,Hmmm ????,Why brag about these cartridges as &#34;CHIP ...,NaN,0.0,hmmm,brag cartridge chip included show ink level hp...,0.0
1371670,1,Didn't last very longâ€¦ and now my printer is...,"Got this a couple weeks ago, direct from Amazo...",NaN,0.0,didnt last very long and now my printer is fin...,got couple week ago direct amazon hp brand new...,0.0
2551306,1,A Lousy Printer,I have this printer on my office desk. I hate ...,NaN,0.0,a lousy printer,printer office desk hate here whybr br take lo...,0.0
2287493,1,Avoid this seller if at all possible,I purchased these in April 2011 as backups for...,NaN,0.0,avoid this seller if at all possible,purchased april backup new nw original black t...,0.0


# TF-IDF Feature Extraction

In [353]:
# def compute_tf_idf_for_feature_extraction(df:pd.DataFrame, col_name: str):
#     """Extract the TF-IDF features from the reviews.

#     Parameters
#     ----------
#     df: `pd.DataFrame`
#         The data
    
#     col_name: `str`
#         Column with reviews

#     Return
#     ------
#     df: `pd.DataFrame`
#         An updated DataFrame with the extra spaces removed
#     """
    
#     feature_vector = []
#     update_text_review = []
#     text_reviews = df[col_name].values

#     vectorizer = TfidfVectorizer(input='content', max_df=1, min_df=1)
    

#     # for text_reviews_idx in range(len(text_reviews)):
#     #     text_review = text_reviews[text_reviews_idx]
#     #     # print("Review: ", text_review)
#     #     update_text_review.append(text_review)
#         # text_review_list = text_review.split(' ')
#         # print("Review: ", text_review_list)

    
#     X = vectorizer.fit_transform(update_text_review)

#     return X
# X = compute_tf_idf_for_feature_extraction(lemmed_df, 'review_body_lower_cased')


In [363]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(lemmed_df['review_body_lower_cased'])

In [364]:
X[0]

<1x101934 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [365]:
y = lemmed_df['sentiment']
y.shape

(200000,)

In [366]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [367]:
X_train.shape, X_test.shape

((160000, 101934), (40000, 101934))

In [368]:
y_train.shape

(160000,)

# Perceptron

In [369]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def evaulate(y_label, y_predicted):
    accuracy = accuracy_score(y_label, y_predicted)
    precision = precision_score(y_label, y_predicted, average='binary')
    recall = recall_score(y_label, y_predicted, average='binary')
    f1 = f1_score(y_label, y_predicted, average='binary')

    return accuracy, precision, recall,f1

clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X_train, y_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

tr_acc, tr_prec, tr_rec, tr_f1 = evaulate(y_train, y_pred_train)
te_acc, te_prec, te_rec, te_f1 = evaulate(y_test, y_pred_test)

print("Training: Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(tr_acc, tr_prec, tr_rec, tr_f1))
print(" Testing: Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(te_acc, te_prec, te_rec, te_f1))


Training: Accuracy: 0.9063, Precision: 0.9289, Recall: 0.8799, F1-Score: 0.9038
 Testing: Accuracy: 0.8509, Precision: 0.8756, Recall: 0.8178, F1-Score: 0.8457


# SVM

# Logistic Regression

# Naive Bayes